In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
from xml.etree import ElementTree as ETree
import pickle
import os

In [2]:
images = glob.glob('data/archive/images/*.png')
annots = glob.glob('data/archive/annotations/*.xml')

# Helper functions

In [3]:
def calculate_iou(box1, box2):
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    # Intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    bb1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    bb2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = float(bb1_area + bb2_area - intersection_area)
    
    iou = intersection_area / union_area
    assert iou >= 0.0
    assert iou <= 1.0
    
    return iou

In [4]:
def read_xml(xml_file):
    gt = []
    root = ETree.parse(xml_file)
    image_name = root.find('filename').text
    for member in root.findall('object'):
        label = member[0].text
        coords = [int(member[5][i].text) for i in range(4)]
        gt.append({
            'label': label,
            'coords': coords
        })
    
    return image_name, gt

# Selective Search Algorithm

In [5]:
data_path = 'data/objects/'

In [6]:
if not os.path.exists(data_path):
    os.makedirs(data_path)

In [149]:
sample_count = {
    'with_mask': {
        'negative': 0,
        'positive': 0,
        'positive_samples': [],
        'negative_samples': []
    },
    'without_mask': {
        'negative': 0,
        'positive': 0,
        'positive_samples': [],
        'negative_samples': []
    },
    'mask_weared_incorrect': {
        'negative': 0,
        'positive': 0,
        'positive_samples': [],
        'negative_samples': []
    }
}
max_samples = 300
max_proposals = 2000

In [150]:
for annot in annots[:1]:
    image_name, gt = read_xml(annot)
    image_path = 'data/archive/images/' + image_name
    img = cv2.imread(image_path)
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchQuality()

    proposal_boxes = ss.process()

    img_copy = img.copy()
    for g in gt:
        label = g['label']
        gt_box = g['coords']
        for boxes in proposal_boxes[:max_proposals]:
            iou = calculate_iou(gt_box, boxes)
            if iou >= 0.5:
                if sample_count[label]['positive'] < max_samples:
                    xmin, ymin, xmax, ymax = boxes
                    sample_count[label]['positive'] += 1
                    sample_count[label]['positve_samples'].append(img[ymin:ymax, xmin:xmax])
            elif iou >= 0.1 and iou < 0.5:
                if sample_count[label]['negative'] < max_samples:
                    xmin, ymin, xmax, ymax = boxes
                    sample_count[label]['negative'] += 1
                    cropped_img = img[ymin:ymax, xmin:xmax]
                    if len(cropped_img) > 0:
                        sample_count[label]['negative_samples'].append(cropped_img)
                    
    if sample_count['with_mask']['negative'] == max_samples \
        and sample_count['with_mask']['positive'] == max_samples \
        and sample_count['without_mask']['positive'] == max_samples \
        and sample_count['without_mask']['negative'] == max_samples \
        and sample_count['mask_weared_incorrect']['positive'] == max_samples \
        and sample_count['mask_weared_incorrect']['negative'] == max_samples:
        break

[222 217  20  50]
[160 141  26  60]
[ 0 12 79 87]
[236 135  30  16]
[221 166  12  17]
[293 123 107  14]
[193 176  50  91]
[373 183  27  22]
[79 74 27 31]
[  0 100  42 167]
[50 82 23 14]
[255 157  40  89]
[ 35 234  29  33]
[372  47  28  31]
[161  83  13  12]
[142  89  20  10]
[225   0  52 101]
[184  78  46  48]
[204   0  23  54]
[330  78  70  20]
[141 153   9  36]
[223 118  34  27]
[307 122  93  15]
[ 30 114  75 119]
[223 215  20  52]
[  0  12  78 108]
[329  84  33  16]
[246 157  18  62]
[103 234  20  33]
[243 200  33  44]
[69 74 47 40]
[ 0 77 38 40]
[159 159  10  43]
[ 4 73 74 22]
[239  72  36  45]
[ 95 102  19  13]
[69 94 52 29]
[262  95  33  30]
[288   0  96  64]
[ 72  95 178 172]
[229  33  24  48]
[269 213  75  54]
[270   0 115  81]
[147   0  18  75]
[59 29 47 78]
[336 179  43  16]
[16 85 48 39]
[  0   0 122 107]
[316 220  79  47]
[ 66 208  14  16]
[ 0 30 57 64]
[208  57  32  32]
[76  0 25 29]
[ 48 230  59  33]
[ 30 120   5  84]
[106 105  20  23]
[  0  98  39 169]
[161  83  12  13]


[110 180  23  16]
[294 138  16  39]
[125 132 275 135]
[194 180  49  87]
[ 92 105  52  30]
[106 105  38  30]
[131 175  15  21]
[255 157  40  88]
[281 213  23  25]
[62 72 65 56]
[ 95 135  11  69]
[ 72  84 328 183]
[264 170  32  61]
[159  82  15  18]
[143 197  16  15]
[212   0  16  28]
[ 0 14 59 31]
[377 193  21  11]
[66 73 13 32]
[ 31 111  84 153]
[100  87  25  17]
[235 134  34  20]
[190  66  18  32]
[144 105  21  27]
[205 156  47  48]
[  0 164  42 103]
[190 105  11  39]
[106 182  28  62]
[59 12 16 33]
[246  53  12  28]
[ 41 101  23  25]
[  0 238  13  29]
[ 8  0 49 40]
[248 202  15  24]
[344  41  41  37]
[210  45  29  31]
[57  0 45 45]
[312 132  88  47]
[  0 162  13 105]
[  0   0 106 113]
[134 191  41  52]
[222 139  41  83]
[377 184  23  20]
[103   0  47 100]
[15 73 69 54]
[206  57  33  35]
[211   0  60  55]
[15 86 54 41]
[133 163  12  50]
[361  51  10  27]
[103   0  35  97]
[138 106  84 139]
[296 160 104  86]
[152  89  29  21]
[194  89  17   9]
[  0 108  34  92]
[112 106  32  29]
[ 56  

[110 180  23  16]
[294 138  16  39]
[125 132 275 135]
[194 180  49  87]
[ 92 105  52  30]
[106 105  38  30]
[131 175  15  21]
[255 157  40  88]
[281 213  23  25]
[62 72 65 56]
[ 95 135  11  69]
[ 72  84 328 183]
[264 170  32  61]
[159  82  15  18]
[143 197  16  15]
[212   0  16  28]
[ 0 14 59 31]
[377 193  21  11]
[66 73 13 32]
[ 31 111  84 153]
[100  87  25  17]
[235 134  34  20]
[190  66  18  32]
[144 105  21  27]
[205 156  47  48]
[  0 164  42 103]
[190 105  11  39]
[106 182  28  62]
[59 12 16 33]
[246  53  12  28]
[ 41 101  23  25]
[  0 238  13  29]
[ 8  0 49 40]
[248 202  15  24]
[344  41  41  37]
[210  45  29  31]
[57  0 45 45]
[312 132  88  47]
[  0 162  13 105]
[  0   0 106 113]
[134 191  41  52]
[222 139  41  83]
[377 184  23  20]
[103   0  47 100]
[15 73 69 54]
[206  57  33  35]
[211   0  60  55]
[15 86 54 41]
[133 163  12  50]
[361  51  10  27]
[103   0  35  97]
[138 106  84 139]
[296 160 104  86]
[152  89  29  21]
[194  89  17   9]
[  0 108  34  92]
[112 106  32  29]
[ 56  

[247   0  20  20]
[295 173  10  33]
[355  41  45  40]
[ 31 134 169 133]
[206  20  40  73]
[293  86 107  51]
[  0   0 117 267]
[150   0 145 207]
[195   0  82 100]
[183  89  33  32]
[ 15  86  98 178]
[294  52 106  66]
[156  99  33  41]
[117  72  32  40]
[185 105  16  40]
[230   0  41  38]
[242 138  26  42]
[135 226  65  41]
[185   0  97 111]
[ 34 172  88  89]
[248 174  19  20]
[276  41 124  80]
[307 132  93  65]
[345 183  55  22]
[292 197  32  24]
[178 138  12  28]
[60 29 42 18]
[ 0  0 59 44]
[ 65 247  29  20]
[218 242  19  25]
[285 132 115 135]
[134 174  40  68]
[69 73 70 55]
[346  41  54  37]
[ 19 113  15  68]
[142   0 145 145]
[284 209  59  58]
[ 7 79 60 23]
[ 6 77 63 41]
[238 108  77  42]
[327 175  73  29]
[244   0  23  20]
[289   0 111  78]
[125   0 275 267]
[362   0  13  38]
[192  27  35  37]
[222 217  20  50]
[160 141  26  60]
[ 0 12 79 87]
[236 135  30  16]
[221 166  12  17]
[293 123 107  14]
[193 176  50  91]
[373 183  27  22]
[79 74 27 31]
[  0 100  42 167]
[50 82 23 14]
[255 1

[ 95 194  21  51]
[  7   0  99 107]
[ 7  0 95 45]
[  0  98 183 169]
[103   0 130 103]
[251 195  12  29]
[136 251  29  16]
[246   0 154 122]
[192   0  37  61]
[  0 172  13  95]
[303  86  51  33]
[332  86  68  36]
[246 174  21  52]
[95 93 31 35]
[134   0  40 100]
[150  80  34  31]
[131 163  19  39]
[ 92  74 144  52]
[344  48  56  43]
[58 12 19 34]
[186  40  43  49]
[184  78  52  48]
[  7  72 119  71]
[303  97  33  22]
[246  57  14  25]
[  0 107  36 118]
[  0   0 106 126]
[266 214  28  30]
[326  41  59  13]
[  0 100 113 167]
[124 190  14  48]
[242 194  51  56]
[ 41 229  23  33]
[98 87 27 17]
[305 109  51  15]
[95 74 42 37]
[ 95 135  30 100]
[304  87  96  37]
[ 60 148  17  15]
[57  0 27 45]
[316  79  47  41]
[289  41 111  37]
[ 0 60 92 66]
[192 168  52  99]
[213   0  64 101]
[182 118  36  82]
[134 188  21  50]
[175 106  47  93]
[302  76  41  41]
[213   0 187 120]
[ 0  0 45 28]
[  0  72 139  71]
[ 51 228  59  39]
[ 3  0 43 26]
[ 30 114 103 153]
[110 180  25  40]
[300  88  68  32]
[117 100  

[108 212  19  32]
[122 214  15  53]
[177 146  10  20]
[28 72 51 34]
[79 56 27 56]
[146  79  86  68]
[304  84  96  38]
[360  31  40  12]
[ 24 113  14  61]
[355   0  27  38]
[ 90 108  53  88]
[258  50  15  33]
[ 0 14 68 64]
[294  87 106 109]
[234  81  21  20]
[150 157   7  24]
[ 65 248  28  19]
[134 188  24  56]
[246  13  27  70]
[183  89  46  19]
[131  87  25  26]
[236 135  32  53]
[103 177  34  68]
[ 34 172  79  91]
[261   0 139 117]
[132 163  75 104]
[  0 238  12  17]
[386  55  14  20]
[275 108  40  39]
[ 55   0  51 107]
[ 97 182  28  44]
[294 136 106  59]
[288  65  22  12]
[ 99 194  12  28]
[ 41 228  54  34]
[182 104  36  96]
[209  88  21  17]
[107 235  30  32]
[158  93  41  53]
[359   0  41  24]
[182  89  29  11]
[249   0 151 137]
[103   0 126 103]
[28 73 54 35]
[179 104  19  42]
[ 35 236  98  31]
[ 34 146  79 117]
[117  74  20  22]
[246 185  17  41]
[ 24 113  15 132]
[168 194  26  26]
[100 102  43  30]
[291 191 109  76]
[300  54  37   9]
[103   0  71  95]
[15 86 54 40]
[ 0 14 68 84

In [9]:
with open(data_path + 'object.pkl', 'wb+') as saved_object:
    pickle.dump(sample_count, saved_object, pickle.HIGHEST_PROTOCOL)